In [12]:
import numpy as np
from scipy import stats
import pandas as pd
from getpass import getpass
import snowflake.connector
password = getpass('Password: ')

In [ ]:
# connect to scowflake
cn = snowflake.connector.connect(
    user = 'mkulczyk',
    password = password,
    account = 'ox58300.eu-west-1',
    warehouse = 'COMPUTE_WH_S',
    database = 'REPORTING',
    schema = 'DWH')

In [ ]:
#total

In [23]:
query_total = '''
with project_details as(
    select PROJECT_ID
          ,PROJECT_PUBLIC_KEY

    from REPORTING.PROJECTS.PROJECT_DETAILS
    where TEST_PROJECT=0 and PROJECT_DELETED_IND=0
 ),

 join_to_convo_from_intents as (
     select distinct
     event_time::date                                    as period_date,
     event_data:"user_properties":"project_id"::integer  as project_id,
      event_type

      from staging.amplitude.stg_amplitude__raw_events
      where event_type ='Conversation: Join to the conversation'
      and
      (event_data:"event_properties":"source"='intents'
       or
       event_data:"event_properties":"source"
       in('shippingPolicy','orderIssues','products'))
       and period_date>='2022-12-01'
 ),
  intent_adoption as (
    select project_id
        , min(period_date) as adoption_date
        , datediff(day,adoption_date,current_date-1) as diff_since_adoption
    from join_to_convo_from_intents
    group by 1
) ,
  ml_convos as (
    select PUBLIC_KEY,
        THREAD_ID,
        CONV_ID
    from staging.NLP.STG_NLP__INTENT_CLASSIFIER_PREDICTIONS
    where INTENT_LABEL in ('order cancelation','order change','order damaged/wrong','order status','order support',
    'product','product exchange','refund','return','shipping change','shipping delivery issue','shipping policy')
),


 conversation_thread as (
    select distinct ct.project_id
                   , ia.adoption_date
                   , ct.started_at::DATE
                   , case when ct.started_at::DATE <adoption_date then 'before' else 'after' end tipping_point
                   , ct.conversation_id
                   , ct.id conversation_thread_id
                   , HAS_MESSAGES_FROM_VISITOR

    from intent_adoption ia
    join project_details pd
    on pd.PROJECT_ID=ia.project_id
    join reporting.core.conversation_thread ct
    on ct.PROJECT_ID=ia.project_id
    join ml_convos mc
     on mc.PUBLIC_KEY=pd.PROJECT_PUBLIC_KEY and  ct.id=mc.THREAD_ID and ct.started_at::DATE  >= dateadd(day,-diff_since_adoption,adoption_date)
     where ia.adoption_date is not null and ct.started_at::DATE>='2022-09-01'
 ),

 conversation_threads_operators as (
    select conversation_thread_id
          , operator_id
          , response_time
          , is_first_response_in_thread

    from reporting.core.conversation_threads_operators
 ),

conversation_threads_joined as (
     select * from conversation_thread
      left join conversation_threads_operators using(conversation_thread_id)
 )

select   project_id,tipping_point
       , sum(response_time) as total_first_response_time
       , round(stddev(RESPONSE_TIME),2) as standard_dev
   -- response_time is null when the convo was started by operator so we want to exclude these threads
       ,  sum(iff(response_time is not null,1,0)) as afrt_conversations_count
       , round(total_first_response_time/afrt_conversations_count,2) avg_frt

from conversation_threads_joined
where has_messages_from_visitor=1 and is_first_response_in_thread=1
group by 1,2
'''

# connect to scowflake
cn = snowflake.connector.connect(
    user = 'mkulczyk',
    password = password,
    account = 'ox58300.eu-west-1',
    warehouse = 'COMPUTE_WH_S',
    database = 'REPORTING',
    schema = 'DWH')

# load the data
data_total= pd.read_sql(query_total, cn)

/Users/magdalenakulczyk/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [24]:
data_total

,PROJECT_ID,TIPPING_POINT,TOTAL_FIRST_RESPONSE_TIME,STANDARD_DEV,AFRT_CONVERSATIONS_COUNT,AVG_FRT
0,1995771,before,784169.0,2408.95,396,1980.22
1,2034556,before,207845.0,3366.63,49,4241.73
2,1448971,before,96637.0,1322.80,256,377.49
3,904563,before,215453.0,1909.58,165,1305.78
4,2134983,before,266774.0,1173.32,743,359.05
...,...,...,...,...,...,...
2272,1886995,after,1756.0,NaN,1,1756.00
2273,1704789,before,150.0,22.63,2,75.00
2274,1832062,before,7770.0,NaN,1,7770.00
2275,2475505,after,7289.0,4770.85,2,3644.50


In [25]:
data_filtered_convo_total = data_total[data_total['AFRT_CONVERSATIONS_COUNT'] >= 2]

In [26]:
data_filtered_convo_total

,PROJECT_ID,TIPPING_POINT,TOTAL_FIRST_RESPONSE_TIME,STANDARD_DEV,AFRT_CONVERSATIONS_COUNT,AVG_FRT
0,1995771,before,784169.0,2408.95,396,1980.22
1,2034556,before,207845.0,3366.63,49,4241.73
2,1448971,before,96637.0,1322.80,256,377.49
3,904563,before,215453.0,1909.58,165,1305.78
4,2134983,before,266774.0,1173.32,743,359.05
...,...,...,...,...,...,...
2262,2379036,before,19525.0,5771.97,4,4881.25
2263,1559627,before,1878.0,1213.40,2,939.00
2270,2545930,before,180.0,86.27,2,90.00
2273,1704789,before,150.0,22.63,2,75.00


In [27]:
pivot_data_total = data_filtered_convo_total.pivot(index='PROJECT_ID', columns='TIPPING_POINT',values='AVG_FRT')
pivot_data_total

TIPPING_POINT,after,before
PROJECT_ID,,
15461,251.50,149.75
23205,1027.00,451.85
54691,1504.00,NaN
61755,106.14,728.83
64855,121.86,107.78
...,...,...
2570326,3107.50,NaN
2571125,2370.00,NaN
2571906,110.00,83.00


In [28]:
filtered_pivot_data_total = pivot_data_total[~pivot_data_total['after'].isna() & ~pivot_data_total['before'].isna()]
filtered_pivot_data_total

TIPPING_POINT,after,before
PROJECT_ID,,
15461,251.50,149.75
23205,1027.00,451.85
61755,106.14,728.83
64855,121.86,107.78
84454,225.63,397.50
...,...,...
2560731,1754.31,2274.50
2561172,2689.50,213.67
2564926,690.25,686.00


In [29]:
stats.ttest_rel(filtered_pivot_data_total.before, filtered_pivot_data_total.after)

Ttest_relResult(statistic=-1.1371165374385863, pvalue=0.2558185340141712)

In [43]:
sum_convo_total  = data_filtered_convo_total.pivot(index='PROJECT_ID', columns='TIPPING_POINT',
                                                       values='AFRT_CONVERSATIONS_COUNT').sum()
sum_time_total  = data_filtered_convo_total.pivot(index='PROJECT_ID', columns='TIPPING_POINT',
                                                      values='TOTAL_FIRST_RESPONSE_TIME').sum()
result_total = (sum_time_total / sum_convo_total ) / 60
result_total

TIPPING_POINT
after     16.157927
before    18.655252
dtype: float64

In [49]:
diff_order_total = result_total.after - result_total.before
diff_order_total

-2.4973253669751827

In [ ]:
#product

In [30]:
query_product = '''
with project_details as(
    select PROJECT_ID
          ,PROJECT_PUBLIC_KEY

    from REPORTING.PROJECTS.PROJECT_DETAILS
    where TEST_PROJECT=0 and PROJECT_DELETED_IND=0
 ),

 join_to_convo_from_intents as (
     select distinct
     event_time::date                                    as period_date,
     event_data:"user_properties":"project_id"::integer  as project_id,
      event_type

      from staging.amplitude.stg_amplitude__raw_events
      where event_type ='Conversation: Join to the conversation'
      and
      (event_data:"event_properties":"source"='intents'
       or
       event_data:"event_properties":"source"
       in('shippingPolicy','orderIssues','products'))
       and period_date>='2022-12-01'
 ),
  intent_adoption as (
    select project_id
        , min(period_date) as adoption_date
        , datediff(day,adoption_date,current_date-1) as diff_since_adoption
    from join_to_convo_from_intents
    group by 1
) ,
  ml_convos as (
    select PUBLIC_KEY,
        THREAD_ID,
        CONV_ID
    from staging.NLP.STG_NLP__INTENT_CLASSIFIER_PREDICTIONS
    where INTENT_LABEL='product'
),


 conversation_thread as (
    select distinct ct.project_id
                   , ia.adoption_date
                   , ct.started_at::DATE
                   , case when ct.started_at::DATE <adoption_date then 'before' else 'after' end tipping_point
                   , ct.conversation_id
                   , ct.id conversation_thread_id
                   , HAS_MESSAGES_FROM_VISITOR

    from intent_adoption ia
    join project_details pd
    on pd.PROJECT_ID=ia.project_id
    join reporting.core.conversation_thread ct
    on ct.PROJECT_ID=ia.project_id
    join ml_convos mc
     on mc.PUBLIC_KEY=pd.PROJECT_PUBLIC_KEY and  ct.id=mc.THREAD_ID and ct.started_at::DATE  >= dateadd(day,-diff_since_adoption,adoption_date)
     where ia.adoption_date is not null and ct.started_at::DATE>='2022-09-01'
 ),

 conversation_threads_operators as (
    select conversation_thread_id
          , operator_id
          , response_time
          , is_first_response_in_thread

    from reporting.core.conversation_threads_operators
 ),

conversation_threads_joined as (
     select * from conversation_thread
      left join conversation_threads_operators using(conversation_thread_id)
 )

select   project_id,tipping_point
       , sum(response_time) as total_first_response_time
       , round(stddev(RESPONSE_TIME),2) as standard_dev
   -- response_time is null when the convo was started by operator so we want to exclude these threads
       ,  sum(iff(response_time is not null,1,0)) as afrt_conversations_count
       , round(total_first_response_time/afrt_conversations_count,2) avg_frt

from conversation_threads_joined
where has_messages_from_visitor=1 and is_first_response_in_thread=1
group by 1,2
'''

# load the data
data_product= pd.read_sql(query_product, cn)

/Users/magdalenakulczyk/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [31]:
data_filtered_convo_product = data_product[data_product['AFRT_CONVERSATIONS_COUNT'] >= 2]
pivot_data_product = data_filtered_convo_product.pivot(index='PROJECT_ID', columns='TIPPING_POINT',values='AVG_FRT')
filtered_pivot_data_product = pivot_data_product[~pivot_data_product['after'].isna() & ~pivot_data_product['before'].isna()]
stats.ttest_rel(filtered_pivot_data_product.before, filtered_pivot_data_product.after)


Ttest_relResult(statistic=-0.5963118156402836, pvalue=0.5511605489701161)

In [42]:
sum_convo_product  = data_filtered_convo_product.pivot(index='PROJECT_ID', columns='TIPPING_POINT',
                                                                values='AFRT_CONVERSATIONS_COUNT').sum()
sum_time_product  = data_filtered_convo_product.pivot(index='PROJECT_ID', columns='TIPPING_POINT',
                                                               values='TOTAL_FIRST_RESPONSE_TIME').sum()
result_product  = (sum_time_product  / sum_convo_product ) / 60
result_product

TIPPING_POINT
after     14.547558
before    15.608034
dtype: float64

In [48]:
diff_order_product = result_product.after - result_product.before
diff_order_product

-1.0604764909340059

In [ ]:
#order status

In [35]:
query_order_status = '''
with project_details as(
    select PROJECT_ID
          ,PROJECT_PUBLIC_KEY

    from REPORTING.PROJECTS.PROJECT_DETAILS
    where TEST_PROJECT=0 and PROJECT_DELETED_IND=0
 ),

 join_to_convo_from_intents as (
     select distinct
     event_time::date                                    as period_date,
     event_data:"user_properties":"project_id"::integer  as project_id,
      event_type

      from staging.amplitude.stg_amplitude__raw_events
      where event_type ='Conversation: Join to the conversation'
      and
      (event_data:"event_properties":"source"='intents'
       or
       event_data:"event_properties":"source"
       in('shippingPolicy','orderIssues','products'))
       and period_date>='2022-12-01'
 ),
  intent_adoption as (
    select project_id
        , min(period_date) as adoption_date
        , datediff(day,adoption_date,current_date-1) as diff_since_adoption
    from join_to_convo_from_intents
    group by 1
) ,
  ml_convos as (
    select PUBLIC_KEY,
        THREAD_ID,
        CONV_ID
    from staging.NLP.STG_NLP__INTENT_CLASSIFIER_PREDICTIONS
    where INTENT_LABEL='order status'
),


 conversation_thread as (
    select distinct ct.project_id
                   , ia.adoption_date
                   , ct.started_at::DATE
                   , case when ct.started_at::DATE <adoption_date then 'before' else 'after' end tipping_point
                   , ct.conversation_id
                   , ct.id conversation_thread_id
                   , HAS_MESSAGES_FROM_VISITOR

    from intent_adoption ia
    join project_details pd
    on pd.PROJECT_ID=ia.project_id
    join reporting.core.conversation_thread ct
    on ct.PROJECT_ID=ia.project_id
    join ml_convos mc
     on mc.PUBLIC_KEY=pd.PROJECT_PUBLIC_KEY and  ct.id=mc.THREAD_ID and ct.started_at::DATE  >= dateadd(day,-diff_since_adoption,adoption_date)
     where ia.adoption_date is not null and ct.started_at::DATE>='2022-09-01'
 ),

 conversation_threads_operators as (
    select conversation_thread_id
          , operator_id
          , response_time
          , is_first_response_in_thread

    from reporting.core.conversation_threads_operators
 ),

conversation_threads_joined as (
     select * from conversation_thread
      left join conversation_threads_operators using(conversation_thread_id)
 )

select   project_id,tipping_point
       , sum(response_time) as total_first_response_time
       , round(stddev(RESPONSE_TIME),2) as standard_dev
   -- response_time is null when the convo was started by operator so we want to exclude these threads
       , sum(iff(response_time is not null,1,0)) as afrt_conversations_count
       , round(total_first_response_time/afrt_conversations_count,2) avg_frt

from conversation_threads_joined
where has_messages_from_visitor=1 and is_first_response_in_thread=1
group by 1,2
'''

# connect to scowflake
cn = snowflake.connector.connect(
    user = 'mkulczyk',
    password = password,
    account = 'ox58300.eu-west-1',
    warehouse = 'COMPUTE_WH_S',
    database = 'REPORTING',
    schema = 'DWH')

# load the data
data_order_status= pd.read_sql(query_order_status, cn)

/Users/magdalenakulczyk/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [36]:
data_filtered_convo_order_status = data_order_status[data_order_status['AFRT_CONVERSATIONS_COUNT'] >= 2]
pivot_data_order_status = data_filtered_convo_order_status.pivot(index='PROJECT_ID', columns='TIPPING_POINT',values='AVG_FRT')
filtered_pivot_data_order_status = pivot_data_order_status[~pivot_data_order_status['after'].isna() & ~pivot_data_order_status['before'].isna()]
stats.ttest_rel(filtered_pivot_data_order_status.before, filtered_pivot_data_order_status.after)

Ttest_relResult(statistic=-0.9455263446386191, pvalue=0.34515882004720466)

In [37]:
sum_convo_order_status = data_filtered_convo_order_status.pivot(index='PROJECT_ID', columns='TIPPING_POINT',values='AFRT_CONVERSATIONS_COUNT').sum()

In [39]:
sum_time_order_status = data_filtered_convo_order_status.pivot(index='PROJECT_ID', columns='TIPPING_POINT',values='TOTAL_FIRST_RESPONSE_TIME').sum()

In [44]:
result_order_status = (sum_time_order_status / sum_convo_order_status)/60

In [45]:
result_order_status

TIPPING_POINT
after     20.932336
before    24.661082
dtype: float64

In [47]:
diff_order_status = result_order_status.after - result_order_status.before
diff_order_status

-3.728746136324549